[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mahdimplus/DeepRetroMoco/blob/main/functions.ipynb)

In [ ]:
# imports
# local imports
import voxelmorph as vxm
import neurite as ne
import os, sys

# third party imports
import numpy as np
import tensorflow as tf
assert tf.__version__.startswith('2.'), 'This tutorial assumes Tensorflow 2.0+'
import matplotlib.pyplot as plt
import cv2

%matplotlib inline

import nbimporter
from functions import *


In [ ]:
data_dir='/home/mahdi/Desktop/data_selection_D7'

In [ ]:
#m=maxx(data_dir)
m=1584 

In [ ]:
split=0.3  # it means 70% for train and 30% for validation


train_generator = data_generator(data_dir,100,m,split)
in_sample, out_sample = next(train_generator)

# visualize
images = [img[0, :, :, 0] for img in in_sample + out_sample] 
titles = ['moving', 'fixed', 'moved ground-truth (fixed)', 'zeros']
ne.plot.slices(images, titles=titles, cmaps=['gray'], do_colorbars=True);


In [ ]:
# configure unet input shape (concatenation of moving and fixed images)
ndim = 2
unet_input_features = 2
# data shape 64*64
s=in_sample[0].shape[1:3]
inshape = (*s, unet_input_features)

# configure unet features 
nb_features = [
    [64, 64, 64, 64],         # encoder features
    [64, 64, 64, 64, 64, 32,16]  # decoder features
]

In [ ]:
s

(64, 64)

In [ ]:
# build model using VxmDense
inshape =s
vxm_model = vxm.networks.VxmDense(inshape, nb_features, int_steps=0)

In [ ]:
vxm_model.summary()

Model: "vxm_dense"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
source_input (InputLayer)       [(None, 64, 64, 1)]  0                                            
__________________________________________________________________________________________________
target_input (InputLayer)       [(None, 64, 64, 1)]  0                                            
__________________________________________________________________________________________________
unet_input_concat (Concatenate) (None, 64, 64, 2)    0           source_input[0][0]               
                                                                 target_input[0][0]               
__________________________________________________________________________________________________
unet_enc_conv_0_0 (Conv2D)      (None, 64, 64, 64)   1216        unet_input_concat[0][0]  

In [ ]:
# voxelmorph has a variety of custom loss classes
losses = [vxm.losses.MSE().loss, vxm.losses.Grad('l2').loss]

# usually, we have to balance the two losses by a hyper-parameter
lambda_param = 0.01
loss_weights = [1, lambda_param]

In [ ]:
vxm_model.compile(optimizer='Adam', loss=losses, loss_weights=loss_weights, metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau,EarlyStopping
es = EarlyStopping(monitor='val_accuracy', mode='max', min_delta=1, patience=5, verbose=1)
#annealer = ReduceLROnPlateau(monitor=['val_accuracy','val_mse'] ,factor=0.5, patience=5, verbose=1, min_lr=1e-3)
checkpoint = ModelCheckpoint('model.h5', verbose=1, save_best_only=True)

In [ ]:
nb_epochs =200
steps_per_epoch =100
hist = vxm_model.fit(train_generator,
                               epochs=nb_epochs,
                               steps_per_epoch=steps_per_epoch,
                               verbose=2,
                               callbacks=[es,checkpoint],
                               validation_data=val_generator(data_dir,1,1584,.3)
                    );

Epoch 1/200

Epoch 00001: val_loss improved from inf to 0.00076, saving model to model.h5
100/100 - 435s - loss: 0.0011 - transformer_loss: 0.0011 - flow_loss: 0.0012 - transformer_accuracy: 0.0000e+00 - flow_accuracy: 0.0607 - val_loss: 7.5929e-04 - val_transformer_loss: 7.3158e-04 - val_flow_loss: 0.0028 - val_transformer_accuracy: 0.0000e+00 - val_flow_accuracy: 0.0000e+00
Epoch 2/200

Epoch 00002: val_loss did not improve from 0.00076
100/100 - 476s - loss: 6.9619e-04 - transformer_loss: 6.6173e-04 - flow_loss: 0.0034 - transformer_accuracy: 0.0000e+00 - flow_accuracy: 0.3390 - val_loss: 8.3998e-04 - val_transformer_loss: 8.0592e-04 - val_flow_loss: 0.0034 - val_transformer_accuracy: 0.0000e+00 - val_flow_accuracy: 0.0061
Epoch 3/200

Epoch 00003: val_loss improved from 0.00076 to 0.00071, saving model to model.h5
100/100 - 485s - loss: 6.3073e-04 - transformer_loss: 5.9271e-04 - flow_loss: 0.0038 - transformer_accuracy: 0.0000e+00 - flow_accuracy: 0.3997 - val_loss: 7.1296e-04 - v

Epoch 14/200

Epoch 00014: val_loss did not improve from 0.00049
100/100 - 469s - loss: 5.3700e-04 - transformer_loss: 4.8095e-04 - flow_loss: 0.0056 - transformer_accuracy: 0.0000e+00 - flow_accuracy: 0.4996 - val_loss: 4.9104e-04 - val_transformer_loss: 4.5049e-04 - val_flow_loss: 0.0041 - val_transformer_accuracy: 0.0000e+00 - val_flow_accuracy: 0.0000e+00
Epoch 15/200

Epoch 00015: val_loss did not improve from 0.00049
100/100 - 472s - loss: 5.3350e-04 - transformer_loss: 4.7685e-04 - flow_loss: 0.0057 - transformer_accuracy: 0.0000e+00 - flow_accuracy: 0.5006 - val_loss: 5.1590e-04 - val_transformer_loss: 4.7459e-04 - val_flow_loss: 0.0041 - val_transformer_accuracy: 0.0000e+00 - val_flow_accuracy: 0.0000e+00
Epoch 16/200

Epoch 00016: val_loss improved from 0.00049 to 0.00047, saving model to model.h5
100/100 - 474s - loss: 5.3017e-04 - transformer_loss: 4.7267e-04 - flow_loss: 0.0057 - transformer_accuracy: 0.0000e+00 - flow_accuracy: 0.4978 - val_loss: 4.7219e-04 - val_transfor

Epoch 27/200

Epoch 00027: val_loss did not improve from 0.00044
100/100 - 474s - loss: 5.1539e-04 - transformer_loss: 4.5147e-04 - flow_loss: 0.0064 - transformer_accuracy: 0.0000e+00 - flow_accuracy: 0.5079 - val_loss: 4.4944e-04 - val_transformer_loss: 4.0033e-04 - val_flow_loss: 0.0049 - val_transformer_accuracy: 0.0000e+00 - val_flow_accuracy: 7.3242e-04
Epoch 28/200

Epoch 00028: val_loss improved from 0.00044 to 0.00043, saving model to model.h5
100/100 - 470s - loss: 5.1664e-04 - transformer_loss: 4.5228e-04 - flow_loss: 0.0064 - transformer_accuracy: 0.0000e+00 - flow_accuracy: 0.5073 - val_loss: 4.2804e-04 - val_transformer_loss: 3.7794e-04 - val_flow_loss: 0.0050 - val_transformer_accuracy: 0.0000e+00 - val_flow_accuracy: 2.4414e-04
Epoch 29/200

Epoch 00029: val_loss improved from 0.00043 to 0.00042, saving model to model.h5
100/100 - 469s - loss: 5.1266e-04 - transformer_loss: 4.4825e-04 - flow_loss: 0.0064 - transformer_accuracy: 0.0000e+00 - flow_accuracy: 0.5064 - val_l

Epoch 40/200

Epoch 00040: val_loss did not improve from 0.00039
100/100 - 491s - loss: 5.0704e-04 - transformer_loss: 4.3976e-04 - flow_loss: 0.0067 - transformer_accuracy: 0.0000e+00 - flow_accuracy: 0.5080 - val_loss: 4.0268e-04 - val_transformer_loss: 3.5180e-04 - val_flow_loss: 0.0051 - val_transformer_accuracy: 0.0000e+00 - val_flow_accuracy: 7.3242e-04
Epoch 41/200

Epoch 00041: val_loss did not improve from 0.00039
100/100 - 492s - loss: 5.0283e-04 - transformer_loss: 4.3567e-04 - flow_loss: 0.0067 - transformer_accuracy: 0.0000e+00 - flow_accuracy: 0.5036 - val_loss: 4.0986e-04 - val_transformer_loss: 3.5412e-04 - val_flow_loss: 0.0056 - val_transformer_accuracy: 0.0000e+00 - val_flow_accuracy: 7.3242e-04
Epoch 42/200

Epoch 00042: val_loss did not improve from 0.00039
100/100 - 490s - loss: 5.0410e-04 - transformer_loss: 4.3668e-04 - flow_loss: 0.0067 - transformer_accuracy: 0.0000e+00 - flow_accuracy: 0.5035 - val_loss: 4.0204e-04 - val_transformer_loss: 3.4980e-04 - val_flo

Epoch 53/200

Epoch 00053: val_loss did not improve from 0.00039
100/100 - 497s - loss: 5.0132e-04 - transformer_loss: 4.3211e-04 - flow_loss: 0.0069 - transformer_accuracy: 0.0000e+00 - flow_accuracy: 0.5022 - val_loss: 3.8654e-04 - val_transformer_loss: 3.3302e-04 - val_flow_loss: 0.0054 - val_transformer_accuracy: 0.0000e+00 - val_flow_accuracy: 7.3242e-04
Epoch 54/200

Epoch 00054: val_loss did not improve from 0.00039
100/100 - 494s - loss: 4.9824e-04 - transformer_loss: 4.2910e-04 - flow_loss: 0.0069 - transformer_accuracy: 0.0000e+00 - flow_accuracy: 0.5067 - val_loss: 4.1528e-04 - val_transformer_loss: 3.5848e-04 - val_flow_loss: 0.0057 - val_transformer_accuracy: 0.0000e+00 - val_flow_accuracy: 7.3242e-04
Epoch 55/200

Epoch 00055: val_loss improved from 0.00039 to 0.00038, saving model to model.h5
100/100 - 494s - loss: 4.9734e-04 - transformer_loss: 4.2775e-04 - flow_loss: 0.0070 - transformer_accuracy: 0.0000e+00 - flow_accuracy: 0.5071 - val_loss: 3.7931e-04 - val_transfor

...

Epoch 144/200

Epoch 00144: val_loss did not improve from 0.00036
100/100 - 495s - loss: 4.8013e-04 - transformer_loss: 4.0463e-04 - flow_loss: 0.0075 - transformer_accuracy: 0.0000e+00 - flow_accuracy: 0.4989 - val_loss: 3.5885e-04 - val_transformer_loss: 3.0705e-04 - val_flow_loss: 0.0052 - val_transformer_accuracy: 0.0000e+00 - val_flow_accuracy: 2.4414e-04
Epoch 145/200

Epoch 00145: val_loss did not improve from 0.00036
100/100 - 492s - loss: 4.8022e-04 - transformer_loss: 4.0449e-04 - flow_loss: 0.0076 - transformer_accuracy: 0.0000e+00 - flow_accuracy: 0.5022 - val_loss: 3.5718e-04 - val_transformer_loss: 3.0291e-04 - val_flow_loss: 0.0054 - val_transformer_accuracy: 0.0000e+00 - val_flow_accuracy: 7.3242e-04
Epoch 146/200

Epoch 00146: val_loss did not improve from 0.00036
100/100 - 495s - loss: 4.8118e-04 - transformer_loss: 4.0538e-04 - flow_loss: 0.0076 - transformer_accuracy: 0.0000e+00 - flow_accuracy: 0.4996 - val_loss: 3.5789e-04 - val_transformer_loss: 3.0513e-04 - val_

Epoch 157/200

Epoch 00157: val_loss did not improve from 0.00035
100/100 - 493s - loss: 4.8099e-04 - transformer_loss: 4.0452e-04 - flow_loss: 0.0076 - transformer_accuracy: 0.0000e+00 - flow_accuracy: 0.4999 - val_loss: 3.5964e-04 - val_transformer_loss: 3.0706e-04 - val_flow_loss: 0.0053 - val_transformer_accuracy: 0.0000e+00 - val_flow_accuracy: 2.4414e-04
Epoch 158/200

Epoch 00158: val_loss did not improve from 0.00035
100/100 - 489s - loss: 4.8031e-04 - transformer_loss: 4.0390e-04 - flow_loss: 0.0076 - transformer_accuracy: 0.0000e+00 - flow_accuracy: 0.5017 - val_loss: 3.5656e-04 - val_transformer_loss: 3.0343e-04 - val_flow_loss: 0.0053 - val_transformer_accuracy: 0.0000e+00 - val_flow_accuracy: 4.8828e-04
Epoch 159/200

Epoch 00159: val_loss improved from 0.00035 to 0.00035, saving model to model.h5
100/100 - 491s - loss: 4.8135e-04 - transformer_loss: 4.0505e-04 - flow_loss: 0.0076 - transformer_accuracy: 0.0000e+00 - flow_accuracy: 0.5031 - val_loss: 3.5247e-04 - val_trans

Epoch 170/200

Epoch 00170: val_loss did not improve from 0.00035
100/100 - 490s - loss: 4.8098e-04 - transformer_loss: 4.0399e-04 - flow_loss: 0.0077 - transformer_accuracy: 0.0000e+00 - flow_accuracy: 0.5003 - val_loss: 3.5714e-04 - val_transformer_loss: 3.0512e-04 - val_flow_loss: 0.0052 - val_transformer_accuracy: 0.0000e+00 - val_flow_accuracy: 2.4414e-04
Epoch 171/200

Epoch 00171: val_loss did not improve from 0.00035
100/100 - 492s - loss: 4.7836e-04 - transformer_loss: 4.0201e-04 - flow_loss: 0.0076 - transformer_accuracy: 0.0000e+00 - flow_accuracy: 0.4982 - val_loss: 3.5550e-04 - val_transformer_loss: 2.9884e-04 - val_flow_loss: 0.0057 - val_transformer_accuracy: 0.0000e+00 - val_flow_accuracy: 2.4414e-04
Epoch 172/200

Epoch 00172: val_loss did not improve from 0.00035
100/100 - 495s - loss: 4.8025e-04 - transformer_loss: 4.0330e-04 - flow_loss: 0.0077 - transformer_accuracy: 0.0000e+00 - flow_accuracy: 0.5055 - val_loss: 3.5537e-04 - val_transformer_loss: 3.0025e-04 - val_

Epoch 183/200

Epoch 00183: val_loss did not improve from 0.00035
100/100 - 495s - loss: 4.7753e-04 - transformer_loss: 4.0032e-04 - flow_loss: 0.0077 - transformer_accuracy: 0.0000e+00 - flow_accuracy: 0.5039 - val_loss: 3.5666e-04 - val_transformer_loss: 3.0189e-04 - val_flow_loss: 0.0055 - val_transformer_accuracy: 0.0000e+00 - val_flow_accuracy: 9.7656e-04
Epoch 184/200

Epoch 00184: val_loss improved from 0.00035 to 0.00035, saving model to model.h5
100/100 - 492s - loss: 4.7944e-04 - transformer_loss: 4.0210e-04 - flow_loss: 0.0077 - transformer_accuracy: 0.0000e+00 - flow_accuracy: 0.5001 - val_loss: 3.4998e-04 - val_transformer_loss: 2.9732e-04 - val_flow_loss: 0.0053 - val_transformer_accuracy: 0.0000e+00 - val_flow_accuracy: 2.4414e-04
Epoch 185/200

Epoch 00185: val_loss did not improve from 0.00035
100/100 - 493s - loss: 4.7936e-04 - transformer_loss: 4.0215e-04 - flow_loss: 0.0077 - transformer_accuracy: 0.0000e+00 - flow_accuracy: 0.4993 - val_loss: 3.5269e-04 - val_trans

Epoch 196/200

Epoch 00196: val_loss improved from 0.00035 to 0.00035, saving model to model.h5
100/100 - 504s - loss: 4.7847e-04 - transformer_loss: 4.0102e-04 - flow_loss: 0.0077 - transformer_accuracy: 0.0000e+00 - flow_accuracy: 0.5096 - val_loss: 3.4993e-04 - val_transformer_loss: 2.9954e-04 - val_flow_loss: 0.0050 - val_transformer_accuracy: 0.0000e+00 - val_flow_accuracy: 2.4414e-04
Epoch 197/200

Epoch 00197: val_loss did not improve from 0.00035
100/100 - 498s - loss: 4.7818e-04 - transformer_loss: 4.0059e-04 - flow_loss: 0.0078 - transformer_accuracy: 0.0000e+00 - flow_accuracy: 0.5067 - val_loss: 3.5064e-04 - val_transformer_loss: 2.9653e-04 - val_flow_loss: 0.0054 - val_transformer_accuracy: 0.0000e+00 - val_flow_accuracy: 7.3242e-04
Epoch 198/200

Epoch 00198: val_loss did not improve from 0.00035
100/100 - 498s - loss: 4.7792e-04 - transformer_loss: 4.0029e-04 - flow_loss: 0.0078 - transformer_accuracy: 0.0000e+00 - flow_accuracy: 0.5060 - val_loss: 3.5470e-04 - val_trans

In [ ]:
vxm_model.save_weights("200epoch_with_val_landa_0.01.h5")

In [ ]:
def plot_history(hist, loss_name='loss'):
    # Simple function to plot training history.
    plt.figure()
    plt.plot(hist.epoch, hist.history[loss_name], '.-')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.show()


In [ ]:
print(history.history.keys())

In [ ]:

plot_history(hist)

In [ ]:
data_dir_TEST='/home/mahdi/Desktop/data_selection_D7'

In [ ]:
# let's get some data
val_generator = data_generator(data_dir_TEST,32,m,.3)
val_input, _ = next(val_generator)

In [ ]:
val_pred = vxm_model.predict(val_input)

In [ ]:
# visualize
images = [img[18, :, :, 0] for img in val_input + val_pred] 
titles = ['moving', 'fixed', 'moved', 'flow']
ne.plot.slices(images, titles=titles, cmaps=['gray'], do_colorbars=True);

In [ ]:
ne.plot.flow([val_pred[1][18].squeeze()], width=10)